In [6]:
import torch
import numpy as np
import os
import argparse
from unet import *
from omegaconf import OmegaConf
from train import trainer
from feature_extractor import * 
from ddad import *
torch.cuda.empty_cache()
def parse_args():
    cmdline_parser = argparse.ArgumentParser('DDAD')    
    cmdline_parser.add_argument('-cfg', '--config', 
                                default= os.path.join(os.path.dirname(os.path.abspath(__file__)),'config.yaml'), 
                                help='config file')
    cmdline_parser.add_argument('--train', 
                                default= True, #False
                                help='Train the diffusion model')
    cmdline_parser.add_argument('--detection', 
                                default= True, #False
                                help='Detection anomalies')
    cmdline_parser.add_argument('--domain_adaptation', 
                                default= True, #False
                                help='Domain adaptation')
    args, unknowns = cmdline_parser.parse_known_args()
    return args

if __name__ == "__main__":
    args = parse_args()
    config = OmegaConf.load(args.config)
    checkpoint_path = os.path.join(os.getcwd(), config.model.checkpoint_dir, "model.pth")
    if not os.path.isfile(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint file not found: {checkpoint_path}")

    checkpoint = torch.load(checkpoint_path)


NameError: name '__file__' is not defined

In [1]:
import torch
import numpy as np
import os
import argparse
from unet import *
from omegaconf import OmegaConf
from train import trainer
from feature_extractor import * 
from ddad import *
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2"

def build_model(config):
    if config.model.DDADS:
        unet = UNetModel(config.data.image_size, 32, dropout=0.3, n_heads=2 ,in_channels=config.data.input_channel)
    else:
        unet = UNetModel(config.data.image_size, 64, dropout=0.0, n_heads=4 ,in_channels=config.data.input_channel)
    return unet

def train(config):
    torch.manual_seed(42)
    np.random.seed(42)
    unet = build_model(config)
    print(" Num params: ", sum(p.numel() for p in unet.parameters()))
    unet = unet.to(config.model.device)
    unet.train()
    unet = torch.nn.DataParallel(unet)
    # checkpoint = torch.load(os.path.join(os.path.join(os.getcwd(), config.model.checkpoint_dir), config.data.category,'1000'))
    # unet.load_state_dict(checkpoint)  
    trainer(unet, config.data.category, config)#config.data.category, 


def detection(config):
    unet = build_model(config)
    checkpoint = torch.load(os.path.join(os.getcwd(), config.model.checkpoint_dir, config.data.category, str(config.model.load_chp)))
    unet = torch.nn.DataParallel(unet)
    unet.load_state_dict(checkpoint)    
    unet.to(config.model.device)
    checkpoint = torch.load(os.path.join(os.getcwd(), config.model.checkpoint_dir, config.data.category, str(config.model.load_chp)))
    unet.eval()
    ddad = DDAD(unet, config)
    ddad()
    

def finetuning(config):
    unet = build_model(config)
    checkpoint = torch.load(os.path.join(os.getcwd(), config.model.checkpoint_dir, config.data.category, str(config.model.load_chp)))
    unet = torch.nn.DataParallel(unet)
    unet.load_state_dict(checkpoint)    
    unet.to(config.model.device)
    unet.eval()
    domain_adaptation(unet, config, fine_tune=True)

c:\anaconda\envs\torch_gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
import argparse
import torch
import numpy as np
from omegaconf import OmegaConf
from unet import UNetModel
from train import trainer
from ddad import DDAD, domain_adaptation

os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2"


def build_model(config):
    """Build UNet model based on the configuration."""
    in_channels = config.data.input_channel
    if config.model.DDADS:
        return UNetModel(config.data.image_size, 32, dropout=0.3, n_heads=2, in_channels=in_channels)
    else:
        return UNetModel(config.data.image_size, 64, dropout=0.0, n_heads=4, in_channels=in_channels)


def train(config):
    """Train the model."""
    torch.manual_seed(config.model.seed)
    np.random.seed(config.model.seed)

    unet = build_model(config)
    print("Number of parameters:", sum(p.numel() for p in unet.parameters()))
    unet = unet.to(config.model.device)
    unet.train()
    unet = torch.nn.DataParallel(unet)

    trainer(unet, config.data.category, config)


def detection(config):
    """Detect anomalies using a pre-trained model."""
    unet = build_model(config)
    checkpoint_path = os.path.join(config.model.checkpoint_dir, config.data.category, str(config.model.load_chp))

    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found at {checkpoint_path}")

    checkpoint = torch.load(checkpoint_path)
    unet = torch.nn.DataParallel(unet)
    unet.load_state_dict(checkpoint)
    unet.to(config.model.device)
    unet.eval()

    ddad = DDAD(unet, config)
    ddad()


def finetuning(config):
    """Fine-tune the model for domain adaptation."""
    unet = build_model(config)
    checkpoint_path = os.path.join(config.model.checkpoint_dir, config.data.category, str(config.model.load_chp))

    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found at {checkpoint_path}")

    checkpoint = torch.load(checkpoint_path)
    unet = torch.nn.DataParallel(unet)
    unet.load_state_dict(checkpoint)
    unet.to(config.model.device)
    unet.eval()

    domain_adaptation(unet, config, fine_tune=True)


def parse_args():
    """Parse command-line arguments."""
    parser = argparse.ArgumentParser(description='DDAD Pipeline')
    parser.add_argument('-cfg', '--config',
                        default=os.path.join(os.path.dirname(os.path.abspath(__file__)), 'config.yaml'),
                        help='Path to the configuration file.')
    parser.add_argument('--train', action='store_true', help='Train the model.')
    parser.add_argument('--detection', action='store_true', help='Perform anomaly detection.')
    parser.add_argument('--domain_adaptation', action='store_true', help='Perform domain adaptation.')
    return parser.parse_args()


if __name__ == "__main__":
    torch.cuda.empty_cache()
    args = parse_args()
    config = OmegaConf.load(args.config)

    print(f"Class: {config.data.category}, w: {config.model.w}, v: {config.model.v}, "
          f"load_chp: {config.model.load_chp}, feature extractor: {config.model.feature_extractor}, "
          f"w_DA: {config.model.w_DA}, DLlambda: {config.model.DLlambda}")

    torch.manual_seed(config.model.seed)
    np.random.seed(config.model.seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(config.model.seed)

    if args.train:
        print('Training...')
        train(config)

    if args.domain_adaptation:
        print('Domain Adaptation...')
        finetuning(config)

    if args.detection:
        print('Detecting Anomalies...')
        detection(config)

NameError: name '__file__' is not defined